# Yatai 

Yatai est un outil qui permet de déployer et opérer des services de Machine Learning sur Kubernetes. 
Il est composé de quatre composants:
- Yatai UI: Une interface utilisateur qui permet de gérer et créer les déploiements.
- Yatai image builder: elle crée des images docker des services
- Yatai deployment: opérateur qui contrôle les déploiements
- bentoml: un package python qui offre une CLI et une librairie

In [ ]:
!pip install -r requirements.txt

On commence par lancer yatai via sspcloud, créer un token et se connecter avec la commande suivante:

In [2]:
!bentoml yatai login --api-token cfrnq833h05c73a6o3ng --endpoint https://yatai.lab.sspcloud.fr

Successfully logged in as user "toto" in organization "default".


## 1- Entrainement du modèle

Un modèle doit être enregisté avec BentoML pour qu'on puisse le déployer avec yatai. Dans ce cas, on a un modèle Sklearn que l'on enregistre sous le bon format en utilisant la fonction (voir `train.py`):
````python
saved_model = bentoml.sklearn.save_model(
    args.model_name,
    model,
    metadata=metadata
)
````
- metadata : c'est un dictionnaire qui contient les métadonnées. Dans cet exemple, on enregistre la précision du modèle.

In [3]:
!python train.py

Model saved: Model(tag="regressor:bhpiufftromnlntx")


On peut lister les modèles enregistrés en utilisant l'interface en ligne de commande:

In [4]:
!bentoml models list

 Tag                         Module           Size      Creation Time       
 regressor:bhpiufftromnlntx  bentoml.sklearn  2.51 MiB  2023-02-23 15:02:06 


Yatai offre un dépot centralisé des modèles. Il est configuré pour qu'il enregistre les modèles sur votre bucket S3. Pour exporter et importer des modèles sur yatai, on utilise les commandes: `bentoml models push/pull <model>`.

In [5]:
!bentoml models push regressor:bhpiufftromnlntx

╭──────────────────────────────────────────────────────────────────────────────╮
╭──────────────────────────────────────────────────────────────────────────────╮
│    0:00:00 Registering model "regressor:bhpiufftromnlntx" with Yatai.. .     │
╭──────────────────────────────────────────────────────────────────────────────╮
│    0:00:00 Creating tar archive for model "regressor:bhpiufftromnlntx".. .   │
╭──────────────────────────────────────────────────────────────────────────────╮
│    0:00:00 Creating tar archive for model "regressor:bhpiufftromnlntx".. ..  │
╭──────────────────────────────────────────────────────────────────────────────╮
│    0:00:00 Creating tar archive for model "regressor:bhpiufftromnlntx".. ..  │
╭──────────────────────────────────────────────────────────────────────────────╮
│    0:00:00 Creating tar archive for model "regressor:bhpiufftromnlntx".. ... │
╭──────────────────────────────────────────────────────────────────────────────╮
│    0:00:01 Creating tar ar

## 2- Création du service

BentoML offre un framework pour créer des services afin de déployer des modèles. 
Un service bentoml est composé des runners et des APIs. 

- Runner: Il s'agit de la méthode d'exécuter l'inférence du modèle. Dans `service.py`, on crée un runner pour le modèle avec: `model_runner = bentoml.sklearn.get("regressor:latest").to_runner()`
- APIs: les APIs définissent comment exposer le service. Un service peut avoir un ou plusieurs APIs. Une API est définie par l'entrée, la sortie et la fonction à exécuter. En décorant une fonction avec `@svc.api`, nous déclarons que la fonction doit être invoquée lorsque cette API est appelée. La fonction API est un endroit idéal pour définir votre logique de service, telle que la récupération de fonctionnalités, le pré et le post-traitement et les inférences de modèle via Runners. En appliquant le service, l'API est transformée en un endpoint HTTP. Dans cet exemple, la fonction `predict` sera exposé via le chemin `/predict`. On peut aussi spécifier le chemin de la fonction dans le décorateur avec: `@svc.api(route='/predict')`.

On peut tester le service créé avec la commande suivante (service fait référence à `service.py`):

In [6]:
!bentoml serve service:svc

2023-02-23T15:04:46+0000 [INFO] [cli] Prometheus metrics for HTTP BentoServer from "service:svc" can be accessed at http://localhost:3000/metrics.
2023-02-23T15:04:47+0000 [INFO] [cli] Starting development HTTP BentoServer from "service:svc" listening on http://0.0.0.0:3000 (Press CTRL+C to quit)
2023-02-23T15:04:59+0000 [INFO] [dev_api_server:regression] 10.233.116.0:35932 (scheme=http,method=GET,path=/,type=,length=) (status=200,type=text/html; charset=utf-8,length=2859) 0.799ms (trace=eb86708cae0bb8bc8f91a5efe171f639,span=1f4b29dc228391aa,sampled=0)
2023-02-23T15:05:00+0000 [INFO] [dev_api_server:regression] 10.233.116.0:35944 (scheme=http,method=GET,path=/static_content/index.css,type=,length=) (status=200,type=text/css; charset=utf-8,length=1125) 32.569ms (trace=9c4ab71ccb417e271a4d273144fe043d,span=81c6c1f6d66c5b26,sampled=0)
2023-02-23T15:05:00+0000 [INFO] [dev_api_server:regression] 10.233.116.0:35932 (scheme=http,method=GET,path=/static_content/swagger-ui.css,type=,length=) (s

Vous pouvez consulter le service en changeant votre url du jupyter comme dans l'exemple suivant:
- si l'url de jupyter est: https://user-mbenxsalha-600538-0.user.lab.sspcloud.fr/ alors l'url du service est: https://user-mbenxsalha-600538-user.user.lab.sspcloud.fr/ (on change le '0' par 'user')

## 3- Construction de Bento

Bento est une archive de fichiers avec tout le code source, les modèles, les fichiers de données et les configurations de dépendance nécessaires pour exécuter un bentoml.Service défini par l'utilisateur, emballé dans un format standardisé.

Alors que bentoml.Service normalise la définition de l'API d'inférence, y compris la logique de service, l'initialisation des runners et l'entrée de l'API, les types de sortie, Bento standardise la manière de reproduire l'environnement requis pour exécuter un bentoml.Service en production.

Un Bento peut être créé avec la commande CLI `bentoml build` et un fichier de construction `bentofile.yaml`.

Le fichier de construction définit les dépendances et le code source nécessaires.

In [7]:
!bentoml build .

Building BentoML service "regression:2cnpbnvtroqnpntx" from build context "/home/onyxia/work/yatai-tutorial".
Packing model "regressor:bhpiufftromnlntx"
Locking PyPI package versions.

██████╗░███████╗███╗░░██╗████████╗░█████╗░███╗░░░███╗██╗░░░░░
██╔══██╗██╔════╝████╗░██║╚══██╔══╝██╔══██╗████╗░████║██║░░░░░
██████╦╝█████╗░░██╔██╗██║░░░██║░░░██║░░██║██╔████╔██║██║░░░░░
██╔══██╗██╔══╝░░██║╚████║░░░██║░░░██║░░██║██║╚██╔╝██║██║░░░░░
██████╦╝███████╗██║░╚███║░░░██║░░░╚█████╔╝██║░╚═╝░██║███████╗
╚═════╝░╚══════╝╚═╝░░╚══╝░░░╚═╝░░░░╚════╝░╚═╝░░░░░╚═╝╚══════╝

Successfully built Bento(tag="regression:2cnpbnvtroqnpntx").


In [8]:
!bentoml list

 Tag                      Size      Creation Time        Path                   
 regression:2cnpbnvtroq…  2.53 MiB  2023-02-23 15:07:58  ~/bentoml/bentos/regr… 


On peut exporter le Bento créé à Yatai:

In [9]:
!bentoml push regression:2cnpbnvtroqnpntx

╭──────────────────────────────────────────────────────────────────────────────╮
╭──────────────────────────────────────────────────────────────────────────────╮
│ Model "regressor:bhpiufftromnlntx" already exists in Yatai, skipping         │
│    0:00:00 Bento repository "regression" not found, creating now.. .         │
╭──────────────────────────────────────────────────────────────────────────────╮
│ Model "regressor:bhpiufftromnlntx" already exists in Yatai, skipping         │
│ Successfully pushed bento "regression:2cnpbnvtroqnpntx"                      │
╰──────────────────────────────────────────────────────────────────────────────╯
Pushing Bento "regression:2cnpbnvtroqnpntx" ━━━━━━ 100.0% • 8.1/8… • ? • 0:00:00
                                                            kB                  


## 4- Déploiement

Une fois le Bento est exporté vers Yatai, il est simple de le déployer via l'interface. Dans la rubrique **deployments** de Yatai, on crée un nouveau et on le configure.
Ensuite, Yatai Image Builder lance automatiquement un pod pour construire et publier une image Docker qui correspond au Bento, en utilisant `kaniko`. (cette étape pourrait prendre quelques minutes).

Une fois l'image est construite et publiée sur docker hub avec succés, le statut du pod de construction (`yatai-bento-image-builder-regression--2-cnpbnvtroqnpntx` dans cet exemple) devient `completed`. Si une erreur survient, le statut serait `error`. 

Enfin, l'image construite et les runners sont importés et lancés dans d'autres pods

In [12]:
!kubectl get pods

NAME                                                      READY   STATUS      RESTARTS   AGE
jupyter-python-600538-0                                   1/1     Running     0          7h19m
regressor-5644967654-qjct8                                3/3     Running     0          72s
regressor-runner-0-6496fd4f78-bgc5x                       2/2     Running     0          73s
vscode-python-581065-0                                    1/1     Running     0          31h
yatai-5ff9f98d6f-wfhbm                                    1/1     Running     0          7h40m
yatai-bento-image-builder-regression--2-cnpbnvtroqnpntx   0/1     Completed   0          10m
yatai-deployment-56586ccdc8-4xfqx                         1/1     Running     0          7h40m
yatai-image-builder-6899769469-td6nw                      1/1     Running     0          7h40m
yatai-postgresql-0                                        1/1     Running     0          7h40m


Yatai crée également un objet kubernetes appelé `service` pour exposer le service bento. Dans cet exemple, le nom du service kube est `regressor`.

In [13]:
!kubectl get service

NAME                                                TYPE        CLUSTER-IP      EXTERNAL-IP   PORT(S)             AGE
jupyter-python-600538                               ClusterIP   None            <none>        8888/TCP,3000/TCP   7h20m
regressor                                           ClusterIP   10.233.59.184   <none>        3000/TCP,3001/TCP   87s
regressor-runner-5efa5688462499626cc1efa9b0639f64   ClusterIP   10.233.1.117    <none>        3000/TCP,3001/TCP   88s
vscode-python-581065                                ClusterIP   None            <none>        8080/TCP            31h
vscode-python-yatai                                 ClusterIP   None            <none>        3000/TCP            30h
yatai                                               ClusterIP   10.233.55.50    <none>        80/TCP              7h41m
yatai-deployment-webhook-service                    ClusterIP   10.233.17.43    <none>        443/TCP             7h41m
yatai-image-builder-webhook-service               

In [14]:
!curl -X POST -H "content-type: application/json" --data "[[5, 3, 2, 31, 40]]" http://regressor:3000/predict

[463514.6666666666]